Copyright (c) 2023 Graphcore Ltd. All rights reserved.

# Dolly 2.0 – The World’s First, Truly Open Instruction-Tuned LLM on IPUs – Inference

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
|   NLP   |  Instruction Finetuned Text Generation  | Dolly 2.0 | N/A | Inference | recommended: 4 |  ???   |

[Dolly 2.0](https://www.databricks.com/blog/2023/04/12/dolly-first-open-commercially-viable-instruction-tuned-llm) is a 12B parameter language model trained and instruction finetuned by [Databricks](https://www.databricks.com). By instruction fine-tuning the large language model (LLM), we obtain an LLM better suited for human interactivity. Crucially, Databricks released all code, model weights, and their fine-tuning dataset with an open-source license that permits commercial use. This makes Dolly 2.0 the world's first, truly open-source instruction-tuned LLM. In this notebook, we will show you how to run Dolly 2.0 using Graphcore IPUs on Paperspace with your own prompts.

In this notebook you will:
- Create and configure a Dolly inference pipeline.
- Run Dolly inference on a text prompts to generate answers to user specified questions.

This notebook requires a minimum of 4 IPUs to run. Support for POD16 is coming soon.

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Environment setup

<!-- TODO: add a run on gradient button. -->
The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you.

[![Run on Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://ipu.dev/t8Jxz1)

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

Run the next cell to install extra requirements for this notebook.

In [ ]:
%pip install -r requirements.txt

In [1]:
import os

number_of_ipus = int(os.getenv("NUM_AVAILABLE_IPU", 4))

## Dolly inference pipeline

Let's begin by loading the inference config for Dolly. A configuration suitable for your instance will automatically be selected.

In [2]:
from utils.setup import dolly_config_setup

config_name = "dolly_pod4"
config, *_ = dolly_config_setup("config/inference.yml", "release", config_name)

2023-04-21 14:15:43 INFO: Starting. Process id: 84067


Next, we want to create our inference pipeline. Here we define the maximum
sequence length. Before executing a model on IPUs it needs to be turned into an
executable format by compiling it. This will happen when the pipeline is
created. All input shapes must be known before compiling, so if the maximum
sequence length is changed, the pipeline will need to be recompiled.

In [3]:
import api

# changing these parameters will trigger a recompile
sequence_length = 256  # max 2048

dolly_pipeline = api.DollyPipeline(
    config,
    sequence_length=sequence_length,
)

2023-04-21 14:15:45 INFO: Creating session
2023-04-21 14:15:45 INFO: Starting PopXL IR construction
2023-04-21 14:17:03 INFO: PopXL IR construction duration: 1.30 mins
2023-04-21 14:17:03 INFO: Starting PopXL compilation
2023-04-21 14:18:34 INFO: PopXL compilation duration: 1.52 mins
2023-04-21 14:18:34 INFO: Downloading 'databricks/dolly-v2-12b' pretrained weights


2023-04-21 14:20:59 INFO: Downloading 'databricks/dolly-v2-12b' tokenizer
2023-04-21 14:20:59 INFO: Starting Loading HF pretrained model to IPU
2023-04-21 14:22:41 INFO: Loading HF pretrained model to IPU duration: 1.70 mins


And run the pipeline!

In [ ]:
dolly_pipeline("How many islands are there in Scotland?")

We've just ran Dolly with the default parameters. Thanks to instruction fine-tuning, we can give prompts to Dolly as if it were a chatbot, as instruction finetuning helps turn a base language model into one more suited for interactive behaviour with humans.

There are a few sampling parameters we can use to control the behaviour of Dolly:
- `temperature` – Indicates whether you want more creative or more factual outputs. A higher value generates more creative outputs and a lower value generates more factual answers. Typical values fall between `0.0` and `1.0`.
- `top_k` – Indicates that only among the highest `top_k` probable tokens should be sampled. Set to 0 to sample across all possible tokens. Must be at minimum `0`, in which case top k sampling is disabled, and at most `config.model.embedding.vocab_size` which is 50,280.
- `output_length` – Indicates the number of tokens to sample before stopping. Sampling can stop early if the model outputs `### END`.
- `print_live` – Indicates whether to see the tokens as they are being sampled. Passing `False` will only return the answer as a string.
- `prompt` – A string containing the question you wish to generate an answer for.

Changing these parameters will not result in any recompilation as the input shape to the model has not changed. These can be freely changed and experimented with in the next cell to produce the behaviour you want from Dolly.

In [7]:
temperature = 0.6
top_k = 5
output_length = None
print_live = True

prompt = "Who was Dolly the sheep?"
answer = dolly_pipeline(
    prompt,
    temperature=temperature,
    k=top_k,
    output_length=output_length,
    print_live=None,
)

2023-04-21 14:29:12 INFO: Attach to IPUs


As Dolly is an instruction fine-tuned model, it was trained with a specific prompt format. Internally, the pipeline will transform your prompts into the correct format. To see the full prompt, you can view the `last_instruction_prompt` attribute on the pipeline:

In [5]:
print(dolly_pipeline.last_instruction_prompt[0])

['The instruction below describes a task. Write a response that appropriately completes the request.\n### Instruction:\nWho was Dolly the sheep?\n### Response:\n']

Remember to detach your pipeline when you are finished to free up resources:

In [6]:
dolly_pipeline.detach()

## Conclusion

In this notebook, we have demonstrated how you can easily run Dolly 2.0 for inference on Graphcore IPUs on text prompts. Instruction fine-tuning is a powerful method of turning a base LLM into one more suited for human interactivity, such as a question-answer model or a chatbot.

Although larger instruction LLMs exist with more world knowledge such as ChatGPT, they are closed-source or are subject to non-commercial licensing. This makes Dolly 2.0 a significant milestone as the first of its kind, with future instruction fine-tuned LLMs no doubt to quickly follow. All of which will be truly open.